In [37]:
import pandas as pd
import os
import numpy as np


def create_dist_mat_from_atsp(path_of_file):
    # specify the path to the .atsp file
    file_path = path_of_file
    print(file_path)


    # initialize variables
    n_nodes = None
    dist_matrix = []

    # open and read the file
    with open(file_path) as f:
        for line in f:
            # extract the number of nodes
            if "DIMENSION" in line:
                n_nodes = int(line.split(":")[1].strip())
            # extract the distance matrix
            elif "EDGE_WEIGHT_SECTION" in line:
                while True:
                    line = f.readline()
                    if "EOF" in line:
                        break
                    dist_matrix.append(list(map(int, line.split())))
            # continue reading until end of file
            elif "EOF" in line:
                break


    #resizing the distance matrix to a square matrix
    list_of_lists = dist_matrix

    flat_list = []
    for sublist in list_of_lists:
        for item in sublist:
            flat_list.append(item)


    # Example 1D list
    lst = flat_list

    # Calculate the size of the square matrix
    n = len(lst)
    size = int(np.sqrt(n))

    # Reshape the 1D list into a 2D numpy array with the desired shape
    arr = np.reshape(lst, (size, size))

    # Print the resulting array
    dist_mat = np.array(arr)

    print("No of cities : "  , n_nodes)
    print(dist_mat.shape)
    return dist_mat

In [40]:
instances_path = os.path.join(os.getcwd() , "ALL_atsp")
instances_files = os.listdir(instances_path)
instances_files2 = [os.path.join(instances_path , i) for i in instances_files if i.endswith(".atsp")]

for i in instances_files2:
    create_dist_mat_from_atsp(i)
    


c:\Users\akshay\Documents\analytics project\AOP_DP_Analytics\AOP_DP_Analytics\AnalyticsProject\LKH-alina\LKH\LKH-Heuristic\ALL_atsp\br17.atsp
No of cities :  17
(17, 17)
c:\Users\akshay\Documents\analytics project\AOP_DP_Analytics\AOP_DP_Analytics\AnalyticsProject\LKH-alina\LKH\LKH-Heuristic\ALL_atsp\ft53.atsp
No of cities :  53
(53, 53)
c:\Users\akshay\Documents\analytics project\AOP_DP_Analytics\AOP_DP_Analytics\AnalyticsProject\LKH-alina\LKH\LKH-Heuristic\ALL_atsp\ft70.atsp
No of cities :  70
(70, 70)
c:\Users\akshay\Documents\analytics project\AOP_DP_Analytics\AOP_DP_Analytics\AnalyticsProject\LKH-alina\LKH\LKH-Heuristic\ALL_atsp\ftv170.atsp
No of cities :  171
(171, 171)
c:\Users\akshay\Documents\analytics project\AOP_DP_Analytics\AOP_DP_Analytics\AnalyticsProject\LKH-alina\LKH\LKH-Heuristic\ALL_atsp\ftv33.atsp
No of cities :  34
(34, 34)
c:\Users\akshay\Documents\analytics project\AOP_DP_Analytics\AOP_DP_Analytics\AnalyticsProject\LKH-alina\LKH\LKH-Heuristic\ALL_atsp\ftv35.atsp
N

In [45]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import time


def create_data_model(path):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = create_dist_mat_from_atsp(path)
    # [
    #     [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
    #     [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
    #     [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
    #     [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
    #     [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
    #     [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
    #     [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
    #     [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
    #     [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
    #     [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
    #     [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
    #     [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
    #     [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
    # ]  # yapf: disable
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)


def main(path):
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model(path)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)



for i in instances_files2:
    start = time.time()
    main(i)
    end = time.time()
    print(end-start , "sec")

c:\Users\akshay\Documents\analytics project\AOP_DP_Analytics\AOP_DP_Analytics\AnalyticsProject\LKH-alina\LKH\LKH-Heuristic\ALL_atsp\br17.atsp
No of cities :  17
(17, 17)
Objective: 39 miles
Route for vehicle 0:
 0 -> 2 -> 13 -> 1 -> 9 -> 10 -> 12 -> 5 -> 15 -> 14 -> 6 -> 3 -> 4 -> 7 -> 8 -> 16 -> 11 -> 0

0.018426179885864258 sec
c:\Users\akshay\Documents\analytics project\AOP_DP_Analytics\AOP_DP_Analytics\AnalyticsProject\LKH-alina\LKH\LKH-Heuristic\ALL_atsp\ft53.atsp
No of cities :  53
(53, 53)
Objective: 8011 miles
Route for vehicle 0:
 0 -> 3 -> 49 -> 52 -> 51 -> 50 -> 48 -> 29 -> 28 -> 25 -> 27 -> 26 -> 7 -> 6 -> 5 -> 9 -> 8 -> 2 -> 36 -> 35 -> 40 -> 37 -> 19 -> 18 -> 15 -> 39 -> 38 -> 4 -> 1 -> 41 -> 43 -> 42 -> 47 -> 46 -> 45 -> 44 -> 34 -> 32 -> 33 -> 31 -> 30 -> 11 -> 10 -> 12 -> 14 -> 13 -> 17 -> 16 -> 23 -> 20 -> 21 -> 24 -> 22 -> 0

0.11296391487121582 sec
c:\Users\akshay\Documents\analytics project\AOP_DP_Analytics\AOP_DP_Analytics\AnalyticsProject\LKH-alina\LKH\LKH-Heuris